In [1]:
import numpy as np
import pandas as pd
import altair as alt
from collections import defaultdict

In [2]:
deck = sorted(
    [ str( i) for i in range( 2, 10)] * 4 + ['10'] * (4*4) + ['A'] * 4
    )
print( deck)

['10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '2', '2', '2', '2', '3', '3', '3', '3', '4', '4', '4', '4', '5', '5', '5', '5', '6', '6', '6', '6', '7', '7', '7', '7', '8', '8', '8', '8', '9', '9', '9', '9', 'A', 'A', 'A', 'A']


In [3]:
current_deck = (deck * 2).copy()

In [4]:
basic_strategy_hard = pd.read_csv('data\\basic-strategy-hard.csv', index_col = 'Index')
# basic_strategy_hard.head()

In [5]:
basic_strategy_soft = pd.read_csv('data\\basic-strategy-soft.csv', index_col = 'Index')
# basic_strategy_soft.head()

In [6]:
basic_strategy_split = pd.read_csv('data\\basic-strategy-split.csv', index_col = 'Index')
# basic_strategy_split.head()

In [7]:
# print( basic_strategy_hard.columns)
# print( basic_strategy_hard.index)

In [8]:
basic_strategy_hard.index = list( map( str, basic_strategy_hard.index))
# print( basic_strategy_hard.columns)
# print( basic_strategy_hard.index)

In [9]:
# print( basic_strategy_soft.columns)
# print( basic_strategy_soft.index)

In [10]:
basic_strategy_soft.index = list( map( str, basic_strategy_soft.index))
# print( basic_strategy_soft.columns)
# print( basic_strategy_soft.index)

In [11]:
def value_sum( cards):

    values = np.array( [], dtype = int)
    num_A = np.count_nonzero( cards == 'A')
    values = np.append( values, sum( list( map( value, cards))))
    for i in range( 1, num_A+1):
        values = np.append( values, sum( list( map( value, cards)))-10*i)
    return values

In [31]:
def HiLo( card):
    Hi = [ '10', 'A']
    Lo = [ '2', '3', '4', '5', '6']
    if card in Hi:
        return -1
    elif card in Lo:
        return 1
    else:
        return 0

In [32]:
def draw_card():
    global count_
    card = np.random.choice( current_deck)
    current_deck.remove( card)
    count_ += HiLo( card)
    return card

def value( card):
    try:
        return int( card)
    except:
        return 11

def check( value):
    if value == 11:
        return 'A'
    else:
        return str( value)

In [33]:
class Dealer():
    def __init__( self, verbose = True):
        self.cards = np.array( [])
        self.value = 0
        self.verbose = verbose
    
    def draw( self):
        card = draw_card()
        self.cards = np.append( self.cards, card)
        self.possible_values = value_sum( self.cards)
        if min( self.possible_values) <= 21:
            self.value = max( self.possible_values[ self.possible_values <= 21])
        else:
            self.value = min( self.possible_values)
        if self.verbose == True:
            print( f'Dealer\'s cards are {self.cards}')

In [34]:
class Player():
    def __init__( self, verbose = True):
        self.cards = np.array( [])
        self.value = 0
        self.verbose = verbose
    
    def draw( self):
        card = draw_card()
        self.cards = np.append( self.cards, card)
        self.possible_values = value_sum( self.cards)
        if min( self.possible_values) <= 21:
            self.value = max( self.possible_values[ self.possible_values <= 21])
        else:
            self.value = min( self.possible_values)
        if verbose == True:
            print( f'Player\'s cards are {self.cards}')

In [35]:
def basic_strategy_action( player, dealer, total):
    if total == 'pair':
        return basic_strategy_split.loc[ str( min( player.cards)), check( dealer.value)]
    elif total == 'hard':
        return basic_strategy_hard.loc[ str( player.value), check( dealer.value)]
    elif total == 'soft':
        return basic_strategy_soft.loc[ str( player.value), check( dealer.value)]

In [130]:
def game( player_upcard = None, dealer_upcard = None):
    global current_deck
    global player
    global dealer
    global count_
    global betsize
    
    surrender = False
    betsize = 1
    
    if count_ == 2:
        betsize = 2
    if count_ == 3:
        betsize = 4
    if count_ == 4:
        betsize = 6
    if count_ >= 5:
        betsize = 8
        
    # print( count_, betsize)
    
    change = 0
    dealer = Dealer( verbose = False)
    player = Player( verbose = False)
    
    if dealer_upcard == None:
        dealer.draw()
    
    if player_upcard == None:
        player.draw()
    else:
        np.append( player.cards, player_upcard) # In case of splitting
        
    player.draw()

    action = 'H'

    while (player.value < 21) and (action != 'S'):
        
        if (len( player.cards) == 2) and min( player.cards) == max( player.cards):
            total = 'pair'
        else:
            if 'A' in player.cards:
                total = 'soft'
            else:
                total = 'hard'

        action = basic_strategy_action( player, dealer, total)

        if action == 'Sur':
            if len( player.cards) == 2:
                surrender = True
                change = -betsize / 2
                return change
            else:
                action = 'H'
        
        if action == 'X':
            splitted_card = min( player.cards)
            # if splitted_card == 'A':
                
            return game( player_upcard = min( player.cards))
        
        if action == 'H':
            player.draw()

        if action == 'D':
            player.draw()
            betsize *= 2
            action = 'S'

    while dealer.value < 17:
        dealer.draw()
    
    if ( (player.value < 22) & (player.value > dealer.value) ) | (dealer.value >= 22):
        change = betsize
    elif (player.value >= 22) | (player.value < dealer.value):
        change = -betsize
    
    if len( current_deck) < 50:
        current_deck = (deck * 2).copy()
        count_ = 0
        # print( 'shuffle')
        
    return change

In [131]:
record_game = {
    'run_num': 0,
    'player_cards': np.array([]),
    'dealer_cards': np.array([]),
    'change': 0,
    'count_': 0
}

In [132]:
record_run = defaultdict( lambda: record_game)

In [133]:
verbose = False

record = defaultdict( lambda: record_run.copy())

for i in range( 20):
    result = np.array( [])
    bankroll = 20
    current_deck = (deck * 2).copy()
    count_ = 0
    
    for j in range( 100):
        
        
        change = game()

        bankroll = bankroll + change
        record_run[ j][ 'run_num'] = j
        record_run[ j][ 'player_cards'] = player.cards
        record_run[ j][ 'dealer_cards'] = dealer.cards
        record_run[ j][ 'betsize'] = betsize
        record_run[ j][ 'change'] = change
        record_run[ j][ 'count_'] = count_
        
        result = np.append( result, bankroll)

        #print( f'Bankroll is {bankroll}.')
        #print()
    record[i] = result

In [134]:
record_melted = pd.DataFrame( record).reset_index().melt( id_vars = 'index', value_vars = list( range( 20)))
record_melted

,index,variable,value
0,0,0,21.0
1,1,0,19.0
2,2,0,27.0
3,3,0,43.0
4,4,0,35.0
...,...,...,...
1995,95,19,-29.0
1996,96,19,-27.0
1997,97,19,-28.0
1998,98,19,-29.0


In [135]:
alt.Chart( record_melted).mark_line().encode(
    x = 'index',
    y = 'value',
    color = 'variable:N')

alt.Chart(...)

# Storing

In [136]:
record_run = defaultdict( lambda: record_game.copy())

verbose = False

record = dict()

result = np.array( [])
bankroll = 20
count_ = 0
current_deck = (deck * 2).copy()
betsize = 1
for j in range( 100):

    change = game()

    bankroll = bankroll + change
    record_run[ j][ 'run_num'] = j
    record_run[ j][ 'player_cards'] = player.cards
    record_run[ j][ 'dealer_cards'] = dealer.cards
    record_run[ j][ 'betsize'] = betsize
    record_run[ j][ 'change'] = change
    record_run[ j][ 'count_'] = count_

    result = np.append( result, bankroll)

    #print( f'Bankroll is {bankroll}.')
    #print()


In [137]:
pd.DataFrame( record_run).T.to_csv( '11-08-0944.csv')

# Old codes

In [128]:
verbose = False

length = []
for i in range( 100):
    current_deck = (deck * 2).copy()
    bankroll = 20
    result = np.array( [])
    while bankroll > 0:
        surrender = False
        betsize = 1
        dealer = Dealer( verbose = False)
        player = Player( verbose = False)

        dealer.draw()
        player.draw()
        player.draw()

        action = 'H'

        while (player.value < 21) and (action != 'S'):
            if 'A' in player.cards:
                total = 'soft'
            else:
                total = 'hard'
                
            action = basic_strategy_action( player, dealer, total)
            
            if action == 'Sur':
                if len( player.cards) == 2:
                    surrender = True
                    break
                else:
                    action = 'H'

            if action == 'H':
                player.draw()

            if action == 'D':
                player.draw()
                betsize *= 2
                action = 'S'

        if surrender == True:
            bankroll -= betsize / 2
            continue

        while dealer.value < 17:
            dealer.draw()

        if len( current_deck) < 50:
            current_deck = (deck * 2).copy()

        if ( (player.value < 22) & (player.value > dealer.value) ) | (dealer.value >= 22):
            bankroll += betsize
        elif (player.value >= 22) | (player.value < dealer.value):
            bankroll -= betsize

        result = np.append( result, bankroll)

        #print( f'Bankroll is {bankroll}.')
        #print()
    length.append( len( result))

In [79]:
if len( current_deck) < 50:
    current_deck = (deck * 2).copy()

In [ ]:
current_deck

In [ ]:
values = np.array( [], dtype = int)
num_A = np.count_nonzero( player.cards == 'A')
values = np.append( values, sum( list( map( value, player.cards))))
for i in range( 1, num_A+1):
    values = np.append( values, sum( list( map( value, player.cards)))-10*i)
values[ values <= 21]

In [ ]:
player.value, dealer.value

In [ ]:
pd.Series( length).describe()

In [ ]:
pd.Series( length).hist( bins = 100)